## Unigram and Bigram Induction
11/13/16 - Implements unigram and bigram induction.
Uses the IMDB dataset folder (http://ai.stanford.edu/~amaas/data/sentiment/).

In [1]:
from sentiment_utils import *
from collections import defaultdict
from nltk import word_tokenize
import math
import pickle as pickle
imdb_folder_location = "../aclImdb" # Change this to wherever your imbd folder is located

/Users/Cathy/anaconda/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Build Bigram and Unigram Maps

In [2]:
## Builds maps over the neighboring contexts. Each token is weighted based on its frequency in that context over the
# training set and its score.
pos_both_neighboring_bigrams = defaultdict(lambda : defaultdict(int))
pos_preceding_bigrams = defaultdict(lambda : defaultdict(int))
pos_following_bigrams = defaultdict(lambda : defaultdict(int))
pos_both_neighboring_unigrams = defaultdict(lambda : defaultdict(int))
for index, (filename, review, score) in enumerate(imdb_sentiment_reader(dataset_type='train', sentiment='pos')):
    tokens = word_tokenize(review.lower())
    for i in range(len(tokens)):
        # Get the preceding bigram if it exists
        preceding_bigram = (tokens[i-2], tokens[i-1]) if i >= 2 else None
        # Get the following bigram if it exists
        following_bigram = (tokens[i+1], tokens[i+2]) if i < len(tokens) - 2 else None
        # Get the preceding and following unigrams if they exist
        preceding_unigram = tokens[i-1] if i >= 1 else None
        following_unigram = tokens[i+1] if i < len(tokens) - 1 else None
        
        # Insert into the appropriate maps
        if preceding_bigram is not None and following_bigram is not None:
            pos_both_neighboring_bigrams[(preceding_bigram, following_bigram)][tokens[i]] += score
        if preceding_bigram is not None:
            pos_preceding_bigrams[preceding_bigram][tokens[i]] += score
        if following_bigram is not None:
            pos_following_bigrams[following_bigram][tokens[i]] += score
        if preceding_unigram is not None and following_unigram is not None:
            pos_both_neighboring_unigrams[(preceding_unigram, following_unigram)][tokens[i]] += score
    if index % 1000 == 0:
        print "Now on: " + str(index)



Now on: 0
Now on: 1000
Now on: 2000
Now on: 3000
Now on: 4000
Now on: 5000
Now on: 6000
Now on: 7000
Now on: 8000
Now on: 9000
Now on: 10000
Now on: 11000
Now on: 12000


In [3]:
## Builds maps over the neighboring contexts. Each token is weighted based on its frequency in that context over the
# training set and its score.
neg_both_neighboring_bigrams = defaultdict(lambda : defaultdict(int))
neg_preceding_bigrams = defaultdict(lambda : defaultdict(int))
neg_following_bigrams = defaultdict(lambda : defaultdict(int))
neg_both_neighboring_unigrams = defaultdict(lambda : defaultdict(int))
for index, (filename, review, score) in enumerate(imdb_sentiment_reader(dataset_type='train', sentiment='neg')):
    tokens = word_tokenize(review.lower())
    for i in range(len(tokens)):
        # Get the preceding bigram if it exists
        preceding_bigram = (tokens[i-2], tokens[i-1]) if i >= 2 else None
        # Get the following bigram if it exists
        following_bigram = (tokens[i+1], tokens[i+2]) if i < len(tokens) - 2 else None
        # Get the preceding and following unigrams if they exist
        preceding_unigram = tokens[i-1] if i >= 1 else None
        following_unigram = tokens[i+1] if i < len(tokens) - 1 else None
        
        # Insert into the appropriate maps
        if preceding_bigram is not None and following_bigram is not None:
            neg_both_neighboring_bigrams[(preceding_bigram, following_bigram)][tokens[i]] -= score
        if preceding_bigram is not None:
            neg_preceding_bigrams[preceding_bigram][tokens[i]] -= score
        if following_bigram is not None:
            neg_following_bigrams[following_bigram][tokens[i]] -= score
        if preceding_unigram is not None and following_unigram is not None:
            neg_both_neighboring_unigrams[(preceding_unigram, following_unigram)][tokens[i]] -= score
    if index % 1000 == 0:
        print "Now on: " + str(index)

Now on: 0
Now on: 1000
Now on: 2000
Now on: 3000
Now on: 4000
Now on: 5000
Now on: 6000
Now on: 7000
Now on: 8000
Now on: 9000
Now on: 10000
Now on: 11000
Now on: 12000


In [4]:
# Combine the dictionaries
both_neighboring_bigrams = {"pos": pos_both_neighboring_bigrams, "neg": neg_both_neighboring_bigrams}
preceding_bigrams = {"pos": pos_preceding_bigrams, "neg": neg_preceding_bigrams}
following_bigrams = {"pos": pos_following_bigrams, "neg": neg_following_bigrams}
both_neighboring_unigrams = {"pos": pos_both_neighboring_unigrams, "neg": neg_both_neighboring_unigrams}

In [5]:
def induction_transform_func(filename, review, score):
    """
    Baseline: returns a review with 'not' inserted in front of any identified adjectives/adverbs.
    """
    def get_best_replacement(words_to_scores, score_type):
        """
        Attempts to find a replacement, but returns None if the replacement is not the correct part of speech
        """
        if score_type == "pos":
            return sorted(words_to_scores, key=words_to_scores.get, reverse=True)[0]
        else:
             return sorted(words_to_scores, key=words_to_scores.get)[0]
    
    score_type = "pos" if score < 7 else "neg" # We want the opposite review type
    upper_tokens = word_tokenize(review)
    tagged_review = nltk.pos_tag(upper_tokens)
    transformed_review = []
    tokens = [token.lower() for token in upper_tokens]
    if len(tokens) != len(tagged_review):
        # Return the original review
        return review
    for i, tagged_word in enumerate(tagged_review):
        # Attempt to find a replacement
        replacement_token = tagged_word[0]
        if tagged_word[1] in ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']:
            # Get all the preceding unigrams_bigrams
            # Get the preceding bigram if it exists
            preceding_bigram = (tokens[i-2], tokens[i-1]) if i >= 2 else None
            # Get the following bigram if it exists
            following_bigram = (tokens[i+1], tokens[i+2]) if i < len(tokens) - 2 else None
            # Get the preceding and following unigrams if they exist
            preceding_unigram = tokens[i-1] if i >= 1 else None
            following_unigram = tokens[i+1] if i < len(tokens) - 1 else None    
            # Try each in turn, checking if we have the right Part of Speech
            if (preceding_bigram, following_bigram) in both_neighboring_bigrams[score_type]:
                replacement_token = get_best_replacement(both_neighboring_bigrams[score_type][(preceding_bigram, following_bigram)], score_type)
            elif (preceding_unigram, following_unigram) in both_neighboring_unigrams[score_type]:
                replacement_token = get_best_replacement(both_neighboring_unigrams[score_type][(preceding_unigram, following_unigram)], score_type)
            elif preceding_bigram in preceding_bigrams[score_type]:
                replacement_token = get_best_replacement(preceding_bigrams[score_type][preceding_bigram], score_type)
            elif following_bigram in following_bigrams[score_type]:
                replacement_token = get_best_replacement(following_bigrams[score_type][following_bigram], score_type)
        transformed_review.append(replacement_token)
        # Only reverse if new word has same POS as old word
    return " ".join(transformed_review)
# Example usage:
for (filename, review, score) in imdb_sentiment_reader(dataset_type='val', sentiment='pos'):
    print "Original review: "
    print review
    print "Transformed review:" 
    transformed = induction_transform_func(filename, review, score)
    print transformed
    break

Original review: 
I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge.
Transformed review:
I went and saw this movie last night after being coaxed to by a few friends of mine . I 'll admit that I was hoping to see it because from what I knew of Ashton Kutche

In [7]:
train_reader = imdb_sentiment_reader(dataset_type='train', sentiment='both')
test_reader = imdb_sentiment_reader(dataset_type='val', sentiment='both')
default_evaluator = DefaultEvaluator(verbose=True)
baseline_runner = ExperimentRunner(train_reader, test_reader, induction_transform_func, 
                               evaluator=default_evaluator, verbose=True)
baseline_runner.run_experiment()

Building positive bigram list...
Now on: 0
Now on: 1000
Now on: 2000
Now on: 3000
Now on: 4000
Now on: 5000
Now on: 6000
Now on: 7000
Now on: 8000
Now on: 9000
Now on: 10000
Now on: 11000
Now on: 12000
Building negative bigram list...
Now on: 0
Now on: 1000
Now on: 2000
Now on: 3000
Now on: 4000
Now on: 5000
Now on: 6000
Now on: 7000
Now on: 8000
Now on: 9000
Now on: 10000
Now on: 11000
Now on: 12000
Now evaluating: 0
Current mean score: nan
Now evaluating: 500
Current mean score: 0.252545553758
Now evaluating: 1000
Current mean score: 0.251717427313
Now evaluating: 1500
Current mean score: 0.239297045012
Now evaluating: 2000
Current mean score: 0.247353615637
Now evaluating: 2500
Current mean score: 0.24774795475
Now evaluating: 3000
Current mean score: 0.289172802129
Now evaluating: 3500
Current mean score: 0.323627150844
Now evaluating: 4000
Current mean score: 0.345581136392
Now evaluating: 4500
Current mean score: 0.36199012089
Finished evaluating 4999 test reviews.
Mean score: 0.

/Users/Cathy/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/Cathy/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
review = "If you had asked me how the movie was throughout the film, I would have told you it was great! However, I left the theatre feeling unsatisfied. After thinking a little about it, I believe the problem was the pace of the ending. I feel that the majority of the movie moved kind of slow, and then the ending developed very fast. So, I would say the ending left me disappointed.<br /><br />I thought that the characters were well developed. Costner and Kutcher both portrayed their roles very well. Yes! Ashton Kutcher can act! Also, the different relationships between the characters seemed very real. Furthermore,I thought that the different plot lines were well developed. Overall, it was a good movie and I would recommend seeing it.<br /><br />In conclusion: Good Characters, Great Plot, Poorly Written/Edited Ending. Still, Go See It!!!"

In [7]:
filename = '10_7.txt'
score = 7

In [8]:
transformed = induction_transform_func(filename, review, score)
paper_evaluator = DefaultEvaluator(verbose=True) # Initialize an evaluator for the paper
print transformed
print "Evaluation score: " + str(paper_evaluator.evaluate(filename, review, transformed, score))


Building positive bigram list...
Now on: 0
Now on: 1000
Now on: 2000
Now on: 3000
Now on: 4000
Now on: 5000
Now on: 6000
Now on: 7000
Now on: 8000
Now on: 9000
Now on: 10000
Now on: 11000
Now on: 12000
Building negative bigram list...
Now on: 0
Now on: 1000
Now on: 2000
Now on: 3000
Now on: 4000
Now on: 5000
Now on: 6000
Now on: 7000
Now on: 8000
Now on: 9000
Now on: 10000
Now on: 11000
Now on: 12000
If you had asked me how the movie was throughout the film , I would have told you it was terrible ! ) , I left the theatre feeling unsatisfied . After thinking a movie about it , I believe the problem was the pace of the ending . I feel that the majority of the movie moved kind of slow , and then the ending is too good . no , I would say the ending left me disappointed. < br / > < br / > I thought that the characters were never developed . Costner and Kutcher both portrayed their roles are good . Yes ! Ashton Kutcher can act ! ) , the sexual relationships between the characters seemed to g